In [ ]:
!pip install keras-tuner

In [3]:
import kerastuner as kt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [4]:
import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv("diabetes.csv")

In [6]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [8]:
data.corr()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
Outcome,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


In [15]:
X = data.drop('Outcome', axis=1)

In [17]:
y = data['Outcome']

In [16]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=48)

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)


In [21]:
X_test = scaler.transform(X_test)

In [25]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam, RMSprop, Adagrad, SGD

In [34]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(8,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer', values = ['RMSprop', 'Adam', 'Adagrad', 'SGD'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [35]:
# Tuner object for KT class
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

In [36]:
tuner.search(X_train, y_train, epochs=10,validation_data=(X_test, y_test))

In [37]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'Adam'}

In [38]:
best_model = tuner.get_best_models(num_models=1)[0]

In [39]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 81
Trainable params: 81
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Will fit the best model now from the Initial_epoch as 11 as we already ran 10 previosly.
best_model.fit(X_train, y_train, epochs=100, initial_epoch=11,validation_data=(X_test, y_test))

Epoch 12/100
20/20 [==============================] - 1s 9ms/step - loss: 0.5598 - accuracy: 0.6987 - val_loss: 0.5331 - val_accuracy: 0.7468
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5523 - accuracy: 0.7068 - val_loss: 0.5274 - val_accuracy: 0.7532
Epoch 14/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5447 - accuracy: 0.7085 - val_loss: 0.5221 - val_accuracy: 0.7468
Epoch 15/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5378 - accuracy: 0.7166 - val_loss: 0.5170 - val_accuracy: 0.7532
Epoch 16/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5309 - accuracy: 0.7231 - val_loss: 0.5126 - val_accuracy: 0.7597
Epoch 17/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5248 - accuracy: 0.7345 - val_loss: 0.5083 - val_accuracy: 0.7532
Epoch 18/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5189 - accuracy: 0.7378 - val_loss: 0.5052 - val_accuracy: 0.7662
Epoch 